In [48]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import os
import openpyxl as px
import numpy as np
import psycopg2 as pg
import configparser

In [3]:
#path credenciais

credenciais_path = r"F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\04. Pessoas\Gabriel Moura\00 - Silvicultura\00 - Credenciais\credenciais.conf"

In [4]:
#credenciais


credenciais = configparser.ConfigParser()

credenciais.read(credenciais_path)

logins = credenciais['credenciais']


conn_dwh = pg.connect(
    user =logins['login'],
    password =logins['senha_sql'],
    host = logins['host'],
    port = logins['port'],
    database = logins['data_base']
)

In [5]:
#path premissas 
path_premissas = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\001. Rolling Forcast - RF\2024\Distribuição Operacional\premissas.xlsx'

In [6]:
#path histórico

path_historico = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\001. Rolling Forcast - RF\2024\Distribuição Operacional\histórico - realizado.xlsx'

In [7]:
#path lista tecnica

path_lista_tecnica = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\001. Rolling Forcast - RF\2024\Distribuição Operacional\20241029_ListasTecnicas-v16.xlsx'

In [8]:
#base histórico realizado
historico = pd.read_excel(path_historico, skiprows=2)

In [9]:
#Capacidade de plantio
cap_plantio = pd.read_excel(path_premissas,sheet_name='cap plantio')

In [10]:
#Encontrando mês operacional

def mes_operacional(df, coluna, nome_coluna):
    # Garantir que a coluna é do tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(df[coluna]):
        df[coluna] = pd.to_datetime(df[coluna], errors='coerce')
    
    
    def calcular_mes_operacional(data):
        
        if pd.isna(data):
            return None 
        
        
        dia = data.day
        mes = data.month
        ano = data.year
        
        
        if dia > 20:
            mes = (mes % 12) + 1
            if mes == 1:
                ano += 1
        
        # Retornar um Timestamp, garantindo que todos são inteiros
        return pd.Timestamp(year=int(ano), month=int(mes), day=1)

    # Aplicar a função para a coluna
    df[nome_coluna] = df[coluna].apply(calcular_mes_operacional)

    return df

In [11]:
#Código de usuário responsável


cursor_usu_resp = conn_dwh.cursor()

cursor_usu_resp.execute(
""" 
select distinct on (cd_usuario)
    dcr_usuario,
    cd_usuario,
    est_reg

from 
    forestry.d_vw_usuarios_perfil
"""

)

#Mesmo o est_reg for apenas 'A' ele considera as pessoas que já não estão mais na empresa


dados_uso_resp = cursor_usu_resp.fetchall()

cursor_usu_resp.close()


colunas_uso_resp = [desc[0] for desc in cursor_usu_resp.description]

#Fechando conexão com o DWH
conn_dwh.close()

uso_resp = pd.DataFrame(dados_uso_resp, columns=colunas_uso_resp)

In [12]:
#Inserindo Supervisor

historico = historico.merge(uso_resp[['dcr_usuario','cd_usuario']],left_on=['cd_usuario_resp'], right_on= ['cd_usuario'], how='left').drop('cd_usuario',axis=1)

In [13]:
#Filtrando somente o que é plantio
historico = historico[historico['Grupo de Atividades'] == 'Plantio']

In [14]:
#De para EPS

de_para_eps = pd.read_excel(path_premissas, sheet_name='eps')

de_para_eps.head(3)

,eps_de,eps_para,regional
0,SOLLUM EMPREEND FLORESTAIS EIRELI EPP,Sollum,CE
1,NILZA MARIA G CAVALHEIRO EPP,Nilza,CE
2,CARPELO SERVICOS FLORESTAIS,Carpelo,NW


In [15]:
#Encontrando Regional JFI

historico['os_eps'] = historico.apply(lambda x : str(x['dcr_ordem_servico'])[8:].split('_')[1].upper(), axis=1)

historico['dcr_fornecedor'] = np.where(historico['dcr_fornecedor'] == 'J F I SILVICULTURA', historico['os_eps'], historico['dcr_fornecedor'])

historico['dcr_fornecedor'] = np.where( (historico['os_eps'] == 'JFI') & (historico['dcr_usuario'] == 'Danilo Roberto Bavaroski') , 'JFICAPÃO', historico['dcr_fornecedor'])

In [ ]:
#retirando do histórico os meses que ainda estão planejados e não estão fechado

historico = historico[(historico['[m] Mês Operacional'] != '2024_11') & (historico['[m] Mês Operacional'] != '2024_12')]

In [17]:
#Atualizando o nome das equipes
historico = historico.merge(de_para_eps, left_on=['dcr_fornecedor'], right_on=['eps_de'],how='left').drop('eps_de',axis=1).rename({'eps_para': 'Equipe de Plantio'},axis=1)

In [18]:
#Volume de plantio agrupado por mês
v_anual = historico.groupby('[m] Mês Operacional')['Soma de [c] Valor de Produção'].sum().reset_index()

In [19]:
#Volume de plantio agrupado por mês e fornecedor
v_anual_eps = historico.groupby(['[m] Mês Operacional','dcr_fornecedor','Equipe de Plantio'])['Soma de [c] Valor de Produção'].sum().reset_index()

#Encontrar proporção de plantio 
v_anual_eps['% de Plantio'] = v_anual_eps['Soma de [c] Valor de Produção'] / v_anual_eps.groupby('[m] Mês Operacional')['Soma de [c] Valor de Produção'].transform('sum')

In [20]:
#Pivot_table

#Visão de volume por eps ao longo dos anos
v_anual_eps_pivot = v_anual_eps.pivot_table(index=['Equipe de Plantio'], columns=['[m] Mês Operacional'], values='Soma de [c] Valor de Produção',aggfunc='sum')

#Visão em percentual por eps ao longo dos anos
perc_anual_eps_pivot = v_anual_eps.pivot_table(index=['Equipe de Plantio'], columns=['[m] Mês Operacional'], values='% de Plantio',aggfunc='sum')

In [ ]:
#último quadrimetre operacional do realizado

last4 = pd.to_datetime(v_anual_eps['[m] Mês Operacional'],format='%Y_%m').max()

last4 = last4 - timedelta(days=120)


In [ ]:
#Base com produção e representatividade do último quadrimestre

#Proporção de planejamento do último quadrimestre

quad_v_anual_eps_pivot = v_anual_eps[pd.to_datetime(v_anual_eps['[m] Mês Operacional'],format='%Y_%m') > last4]

In [59]:
#Área referência para distribuição de blocos

area_ref = pd.read_excel(path_premissas,sheet_name='area referencia')

#Retirando áreas de exaustão

area_ref = area_ref[area_ref['Tipo'] != 'Exaustão']


area_ref = area_ref.groupby(['Regional','Mês'])['Área'].sum().reset_index()

In [60]:
area_ref

,Regional,Mês,Área
0,Global,2020-01-01,3128.57
1,Global,2020-02-01,3352.17
2,Global,2020-03-01,1290.72
3,Global,2020-04-01,1725.23
4,Global,2020-05-01,2062.73
...,...,...,...
79,Global,2026-08-01,3125.00
80,Global,2026-09-01,3125.00
81,Global,2026-10-01,3125.00
82,Global,2026-11-01,3125.00
